In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy 
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
consumer_secret = 'GgRmaaK6eG4Hpm8HRAWQdxS5lwUGOSUx0UtAuNfAVDlsY5cpI1'
access_token = '514398705-xz4jxeIkgEhe6iHeRApySDgr7VNTKWaMWProInSf'
consumer_key = 'yBbCjOMnQnOj15GDOOt6LQftP'
access_token_secret = 'kbzXLHyYF6EYaTPG6JzJAAf2gor0RgVd2yzQoUU4bAgl3'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [3]:
df = pd.read_csv('twitter-archive-enhanced.csv')
df.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [27]:
def load_tweets():
    tweets_found = []
    tweets_not_found = []
    
    for tweet_id in df['tweet_id']:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json_content = tweet._json        

            tweets_found.append(json_content)
        except Exception as e:
            tweets_not_found.append(tweet_id)
            
    print('Number of Tweets found: {}.\nNumber of Tweets not found: {}.'.format(len(tweets_found), len(tweets_not_found)))
    return tweets_found

# tweets_found = load_tweets()

Rate limit reached. Sleeping for: 387
Rate limit reached. Sleeping for: 380


Number of Tweets found: 2337.
Number of Tweets not found: 19.


In [61]:
def write_tweets():
    with open('tweet_json.txt', 'w', encoding='utf-8') as outfile:
        outfile.write('[')
        for tweet in tweets_found:
            json.dump(tweet, outfile)
            
            if tweet is not tweets_found[-1]:
                outfile.write(',\n')
            else:
                outfile.write('')
        outfile.write(']')
        
# write_tweets()

In [77]:
with open('tweet_json.txt', encoding='utf-8') as json_file:  
    tweets = json.load(json_file)

In [83]:
nm = 0
index = 0
for tweet in tweets:
    if 'retweeted_status' in dict(tweet):
        nm += 1
        tweets[index] = tweet['retweeted_status']
    index += 1
nm

0

In [91]:
tweets_dict = []

for tweet in tweets:
    tweets_dict.append({'tweet_id': tweet['id'],
                         'favorite_count': tweet['favorite_count'],
                         'retweet_count': tweet['retweet_count'],
                        })

tweet_df = pd.DataFrame(tweets_dict, columns = ['tweet_id', 'favorite_count','retweet_count'])
tweet_df.head(5)

,tweet_id,favorite_count,retweet_count
0,892420643555336193,37516,8178
1,892177421306343426,32261,6052
2,891815181378084864,24300,4004
3,891689557279858688,40852,8326
4,891327558926688256,39069,9026


In [89]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 3 columns):
tweet_id          2337 non-null int64
favorite_count    2337 non-null int64
retweet_count     2337 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB
